In [50]:
import yfinance as yf
from langchain.tools import BaseTool
from typing import Optional, Type
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI



def get_stock_price(symbol):
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return round(todays_data['Close'][0], 2)

get_stock_price('GOOG')

123.02

In [51]:
functions = [
    {
        "name":"get_current_weather",
        "description":"Get current weather for a city",
        "parameters": {
            "type":"object",
            "properties":{
                "location":{
                    "type":"string",
                    "description":"City name and state code (e.g. 'San Francisco, CA')",
                },
                "format":{
                    "type":"string",
                    "enum":["celcius","fahrenheit"],
                    "description":"Temperature format. infer this from users location",
                },
            },
            "required":["location", "format"],
        },
    }
]

In [80]:
from typing import Any
from pydantic import BaseModel, Field

class StockPriceCheckInput(BaseModel):
    stockTicker: str = Field(..., description="Stock ticker to check price for")


class StockPriceTool(BaseTool):
    name = "get_stock_price"
    description = "Useful for when you need to find out the price of a stock. You shoud input the stock ticker used "

    def _run(self, stockTicker: str):
        return get_stock_price(stockTicker)
    def _arun(self, stockTicker: str):
        raise NotImplementedError("This tool does not support async")
    
    args_schema: Optional[Type[BaseModel]] = StockPriceCheckInput


In [81]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage, FunctionMessage


In [82]:
model = ChatOpenAI(model="gpt-3.5-turbo-0613")

In [83]:
from langchain.tools import MoveFileTool, format_tool_to_openai_function

{'name': 'move_file',
  'description': 'Move a file from one location to another',
    'parameters': {'title': 'FileMoveInput',
                   'description': 'Input for MoveFileTool',
                   'type': 'object',
                   'properties': {'source_path': {'title': 'Source Path',
                                                  'description': 'Path of the file to move',
                                                  'type': 'string'},
                                    'destination_path': {'title': 'Destination Path',
                                                            'description': 'Path to move the file to',
                                                            'type': 'string'}},
                     'required': ['source_path', 'destination_path']}}



{'name': 'move_file',
 'description': 'Move a file from one location to another',
 'parameters': {'title': 'FileMoveInput',
  'description': 'Input for MoveFileTool',
  'type': 'object',
  'properties': {'source_path': {'title': 'Source Path',
    'description': 'Path of the file to move',
    'type': 'string'},
   'destination_path': {'title': 'Destination Path',
    'description': 'Path to move the file to',
    'type': 'string'}},
  'required': ['source_path', 'destination_path']}}

In [84]:
tools = [StockPriceTool()]
functions = [format_tool_to_openai_function(tool) for tool in tools]

functions[0]

{'name': 'get_stock_price',
 'description': 'Useful for when you need to find out the price of a stock. You shoud input the stock ticker used ',
 'parameters': {'title': 'StockPriceCheckInput',
  'type': 'object',
  'properties': {'stockTicker': {'title': 'Stockticker',
    'description': 'Stock ticker to check price for',
    'type': 'string'}},
  'required': ['stockTicker']}}

In [85]:
ai_message = model.predict_messages([HumanMessage(content='What is the price of the tesla stock?')], functions=functions)


In [86]:
ai_message
ai_message.additional_kwargs['function_call']

{'name': 'get_stock_price', 'arguments': '{\n  "stockTicker": "TSLA"\n}'}

In [87]:
import json

_args = json.loads(ai_message.additional_kwargs['function_call'].get('arguments'))
_args

{'stockTicker': 'TSLA'}

In [88]:
tool_result = tools[0](_args)
tool_result

256.6

In [89]:
FunctionMessage(name='get_stock_price', content=tool_result)

FunctionMessage(content='256.6', additional_kwargs={}, name='get_stock_price')

In [90]:
final_message = model.predict_messages([HumanMessage(content='What is the price of the tesla stock?'),
                                         ai_message,
                                         FunctionMessage(name='get_stock_price', content=tool_result),
                                         ], functions=functions)

In [91]:
final_message

AIMessage(content='The current price of Tesla stock is $256.6.', additional_kwargs={}, example=False)

In [92]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

In [93]:
open_ai_agent = initialize_agent(tools,
                                 llm,
                                 agent=AgentType.OPENAI_FUNCTIONS,
                                 verbose=True)

In [94]:
open_ai_agent.run("What is the price of the tesla stock?")



> Entering new  chain...

Invoking: `get_stock_price` with `{'stockTicker': 'TSLA'}`


256.6The current price of Tesla stock is $256.6.

> Finished chain.


'The current price of Tesla stock is $256.6.'

GUIDE TO CREATE A TOOL THEN AGENT

1. define methods
2. create the tools
3. initilize agent

In [95]:
from datetime import datetime, timedelta

def get_price_change_percentage(symbol, days_ago):
    ticker = yf.Ticker(symbol)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_ago)

    historical_data = ticker.history(start=start_date, end=end_date)

    old_price = historical_data['Close'].iloc[0]
    new_price = historical_data['Close'].iloc[-1]

    percent_change = ((new_price - old_price) / old_price) * 100
    return round(percent_change, 2)


def calculate_performance(symbol, days_ago):
    ticker = yf.Ticker(symbol)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_ago)
    start_date = start_date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")
    historical_data = ticker.history(start=start_date, end=end_date)
    old_price = historical_data['Close'].iloc[0]
    new_price = historical_data['Close'].iloc[-1]
    percent_change = ((new_price - old_price) / old_price) * 100
    return round(percent_change, 2)


def get_best_performing_stock(stocks, days_ago):
    best_stock = None
    best_performance = None

    for stock in stocks:
        try:
            performance = calculate_performance(stock, days_ago)
            if best_performance is None or performance > best_performance:
                best_performance = performance
                best_stock = stock
        except Exception as e:
            print(f"could not get performance for {stock}: {e}")
    return best_stock, best_performance


stocks = ['AAPL', 'MSFT', 'GOOG' ]
days_ago=30

best_stock, best_performance = get_best_performing_stock(stocks, days_ago)
best_stock, best_performance

('AAPL', 5.29)

In [96]:
from typing import List

class StockChangePercentageCheckInput(BaseModel):
    """input for Stock ticker check. for percentage check"""
    stockTicker: str = Field(..., description="Stock ticker to check price for")
    days_ago: int = Field(..., description="Number of days ago to check price for")


class StockPercentageChangeTool(BaseTool):
    name = "get_price_change_percentage"
    description = "Useful when you need to find the percentage change of a stock over a period of time"

    def _run(self, stockTicker: str, days_ago: int):
        return get_price_change_percentage(stockTicker, days_ago)
    def _arun(self, stockTicker: str, days_ago: int):
        raise NotImplementedError("This tool does not support async")
    
    args_schema: Optional[Type[BaseModel]] = StockChangePercentageCheckInput


class StockBestPerformingInput(BaseModel):
    """input for Stock ticker check. for percentage check"""
    stockTickers: List[str] = Field(..., description="Stock tickers to check price for")
    days_ago: int = Field(..., description="Number of days ago to check price for")

class StockGetBestPerformingTool(BaseTool):
    name = "get_best_performing_stock"
    description = "Useful when you need to find the best performing stock over a period of time"

    def _run(self, stockTickers: List[str], days_ago: int):
        return get_best_performing_stock(stockTickers, days_ago)
    def _arun(self, stockTickers: List[str], days_ago: int):
        raise NotImplementedError("This tool does not support async")
    
    args_schema: Optional[Type[BaseModel]] = StockBestPerformingInput


tools = [StockPriceTool(), StockPercentageChangeTool(), StockGetBestPerformingTool()]



In [97]:
tools

[StockPriceTool(name='get_stock_price', description='Useful for when you need to find out the price of a stock. You shoud input the stock ticker used ', args_schema=<class '__main__.StockPriceCheckInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False),
 StockPercentageChangeTool(name='get_price_change_percentage', description='Useful when you need to find the percentage change of a stock over a period of time', args_schema=<class '__main__.StockChangePercentageCheckInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False),
 StockGetBestPerformingTool(name='get_best_performing_stock', description='Useful when you need to find the best performing stock over a period of time', args_schema=<class '__main__.StockBestPerformingInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False)]

In [98]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")


In [99]:
open_ai_agent = initialize_agent(tools,
                                    llm,
                                    agent=AgentType.OPENAI_FUNCTIONS,
                                    verbose=True)

In [100]:
open_ai_agent.run("What is the price of the tesla stock?")



> Entering new  chain...

Invoking: `get_stock_price` with `{'stockTicker': 'TSLA'}`


256.6The current price of Tesla stock is $256.6.

> Finished chain.


'The current price of Tesla stock is $256.6.'

In [101]:
open_ai_agent.run("Has ggoogles stock gone up over the last 30 days?")



> Entering new  chain...

Invoking: `get_price_change_percentage` with `{'stockTicker': 'GOOGL', 'days_ago': 30}`


-1.08No, Google's stock (GOOGL) has decreased by 1.08% over the last 30 days.

> Finished chain.


"No, Google's stock (GOOGL) has decreased by 1.08% over the last 30 days."

In [184]:
import requests

def get_sport_events_by_date(sport, league, days, teams):

    if days > 0:
        dste = datetime.now() + timedelta(days=days)
    else:
        dste = datetime.now() - timedelta(days=abs(days))

    date = dste.strftime("%Y%m%d")

    url = "https://site.api.espn.com/apis/site/v2/sports/" + sport + "/" + league + "/scoreboard?dates=" + date
    response = requests.get(url)
    json_data = response.json()
    
    events = json_data['events']
    relevant_data = []
    for event in events:
        team1 = event['competitions'][0]['competitors'][0]['team']['displayName']
        team2 = event['competitions'][0]['competitors'][1]['team']['displayName']  

        specific_keys = {
            "event": event.get("name"),
            "isFinished": event["status"]["type"].get('completed'),
            "when": event.get("date"),
            "id": event.get("id"),
            "headlines": event.get("headlines"),
            "venue": event['competitions'][0].get("venue"),
            "format": event['competitions'][0].get("format"),
            "notes": event['competitions'][0].get("notes"),
        }

        if (team1 or team2 in teams) or teams == None:
            relevant_data.append(specific_keys)
    
    return relevant_data




In [185]:
class SportEventsByDateCheckInput(BaseModel):
    """input for get sport events by date check"""
    sport: str = Field(..., description="Sport to check events for.... given a league sport should be the coressponding sport")
    league: str = Field(..., description="League to check events for.... given the sport league should be the most common league for the sport in the united states")
    days: int = Field(..., description="representing the number of days from today to check events for. negative numbers are in the past, positive numbers are in the future")
    teams: List[str] = Field(..., description="List of teams to filter events by. If none are provided all events will be returned and this list should be empty the team names should be written in there full display name ex. 'Los Angeles Lakers'")
class SportEventsByDateTool(BaseTool):
    name = "get_sports_events_by_date"
    description = "Useful when you need information on the games for a particular sport League in the undited states ex. leagyes = mlb,nfl,nba,nhl,mls,wnba, mens-college-basketball, womens-college-basketball, mens-college-football"

    def _run(self, sport: str, league: str, days: int, teams: List[str]):
        return get_sport_events_by_date(sport, league, days, teams)
    def _arun(self, sport: str, league: str, days: int, teams: List[str]):
        raise NotImplementedError("This tool does not support async")
    
    args_schema: Optional[Type[BaseModel]] = SportEventsByDateCheckInput
tools = [SportEventsByDateTool()]

In [186]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
open_ai_agent = initialize_agent(tools,
                                    llm,
                                    agent=AgentType.OPENAI_FUNCTIONS,
                                    verbose=True)

In [189]:
open_ai_agent('can you give me a baseball game to wager on that starts in the afternoon')



> Entering new  chain...

Invoking: `get_sports_events_by_date` with `{'sport': 'baseball', 'league': 'mlb', 'days': 1, 'teams': [], 'time': 'afternoon'}`


[{'event': 'Cincinnati Reds at Baltimore Orioles', 'isFinished': False, 'when': '2023-06-27T23:05Z', 'id': '401472197', 'headlines': None, 'venue': {'id': '1', 'fullName': 'Oriole Park at Camden Yards', 'address': {'city': 'Baltimore', 'state': 'Maryland'}, 'capacity': 45971, 'indoor': True}, 'format': {'regulation': {'periods': 9}}, 'notes': []}, {'event': 'San Diego Padres at Pittsburgh Pirates', 'isFinished': False, 'when': '2023-06-27T23:05Z', 'id': '401472209', 'headlines': None, 'venue': {'id': '47', 'fullName': 'PNC Park', 'address': {'city': 'Pittsburgh', 'state': 'Pennsylvania'}, 'capacity': 38753, 'indoor': False}, 'format': {'regulation': {'periods': 9}}, 'notes': []}, {'event': 'San Francisco Giants at Toronto Blue Jays', 'isFinished': False, 'when': '2023-06-27T23:07Z', 'id': '401472206', 'headlines': None, 'venue': 

{'input': 'can you give me a baseball game to wager on that starts in the afternoon',
 'output': 'Here are some baseball games that start in the afternoon:\n\n1. Cincinnati Reds at Baltimore Orioles - Starts at 4:05 PM ET\n2. San Diego Padres at Pittsburgh Pirates - Starts at 4:05 PM ET\n3. San Francisco Giants at Toronto Blue Jays - Starts at 4:07 PM ET\n4. Miami Marlins at Boston Red Sox - Starts at 4:10 PM ET\n5. Milwaukee Brewers at New York Mets - Starts at 4:10 PM ET\n\nPlease note that the game times are in Eastern Time (ET).'}